In [36]:
def predict_crop_yield(state, district, crop, year, month):
    # Ensure numeric values are integers
    year = int(year)
    month = int(month)

    # Build input dataframe
    user_input = pd.DataFrame([{
        "State": state.strip(),
        "District": district.strip(),
        "Crop": crop.strip(),
        "Crop_Year": year,
        "Month": month
    }])

    # Debugging info
    print("\n[DEBUG] User input DataFrame:")
    print(user_input.dtypes)
    print(user_input)

    # Predict with trained model
    predicted_yield = crop_yield_model.predict(user_input)
    return predicted_yield[0]


In [12]:
import joblib

# Load crop yield prediction pipeline
crop_yield_model = joblib.load('crop_yield_model.pkl')

# Load historical weather model
historical_weather_model = joblib.load('historical_weather_model.pkl')

def predict_weather_historical(lat, lon, month, year):
    # Use the loaded historical model to predict weather
    # Prepare input features the same way as model training
    # Example (replace with your feature engineering):
    X = [[lat, lon, month, year]]  # Adjust features as per your model
    prediction = historical_weather_model.predict(X)
    # Suppose prediction returns [temperature, precipitation] or similar
    # Return dict accordingly (update based on your model output)
    return {
        'predicted_temperature': prediction[0][0],
        'predicted_precipitation_mm': prediction[0][1]
    }


In [13]:
def predict_weather_historical(lat, lon, month, year):
    X = [[lat, lon, month, year]]
    prediction = historical_weather_model.predict(X)
    return {
        'predicted_temperature': prediction[0][0],
        'predicted_precipitation_mm': prediction[0][1]
    }


In [27]:
def integrated_prediction(state, district, crop, year, month):
    # Get crop yield (pass month also)
    crop_yield = predict_crop_yield(state, district, crop, year, month)

    # Get weather (historical model)
    weather = predict_weather_historical(month, year)

    return {
        "Predicted Crop Yield": crop_yield,
        "Predicted Temperature (°C)": weather['predicted_temperature'],
        "Predicted Precipitation (mm)": weather['predicted_precipitation_mm']
    }


In [30]:
state = input("Enter State: ")
district = input("Enter District: ")
crop = input("Enter Crop: ")
year = int(input("Enter Year: "))
month = int(input("Enter Month (1-12): "))

output = integrated_prediction(state, district, crop, year, month)


TypeError: '<' not supported between instances of 'int' and 'str'

In [6]:
import requests
import joblib
import pandas as pd

API_KEY = "mTgxFT8btimLVUoCY3ZkKQiwPxP6k0d6"

# Load the fallback weather model (trained on historical data)
weather_model = joblib.load("weather_model.pkl")

def get_weather_forecast(state, district, year, month):
    """Fetch weather forecast using Tomorrow.io API.
    If API fails, fallback to ML-based prediction from historical data.
    """
    try:
        # Call Tomorrow.io API
        url = f"https://api.tomorrow.io/v4/weather/forecast?location={district},{state}&apikey={API_KEY}"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        forecast = data["timelines"]["daily"][0]  # pick the first day's forecast
        return {
            "source": "Tomorrow.io API",
            "date": forecast["time"],
            "temperature": forecast["values"]["temperatureAvg"],
            "precipitation": forecast["values"]["precipitationSum"],
            "humidity": forecast["values"]["humidityAvg"]
        }

    except Exception:
        # Fallback to trained ML model (historical prediction)
        features = pd.DataFrame([{
            "State": state,
            "District": district,
            "Year": int(year),
            "Month": int(month)
        }])
        pred = weather_model.predict(features)[0]
        return {
            "source": "Historical Model",
            "temperature": pred[0],
            "precipitation": pred[1],
            "humidity": pred[2]
        }
